In [18]:
import pandas as pd
import ydata_profiling

In [19]:
pd.set_option('display.max_columns', None)

In [20]:
results = pd.read_csv(r"data/results.csv")
races = pd.read_csv(r"data/races.csv")
drivers = pd.read_csv(r"data/drivers.csv")
driver_standings = pd.read_csv(r"data/driver_standings.csv")
constructors = pd.read_csv(r"data/constructors.csv")
constructor_standings = pd.read_csv(r"data/constructor_standings.csv")
constructor_results = pd.read_csv(r"data/constructor_results.csv")
pit_stops = pd.read_csv(r"data/pit_stops.csv")
status = pd.read_csv(r"data/status.csv")
seasons = pd.read_csv(r"data/seasons.csv")

circuits = pd.read_csv(r"data/circuits.csv")
lap_times = pd.read_csv(r"data/lap_times.csv")
qualifying = pd.read_csv(r"data/qualifying.csv")
sprint_results = pd.read_csv(r"data/sprint_results.csv")

In [21]:
print('\nDrivers:')
print((drivers == "\\N").sum())
print('\nRaces:')
print((races == "\\N").sum())
print('\nResults:')
print((results == "\\N").sum())
print('\nConstructor_results:')
print((constructor_results == "\\N").sum())
print('\nQualifying:')
print((qualifying == "\\N").sum())
print('\nSprint_results:')
print((sprint_results == "\\N").sum())

# W pozostałych zbiorach nie ma pustych wartości:
# print((circuits == "\\N").sum())
# print((constructor_standings == "\\N").sum())
# print((constructors == "\\N").sum())
# print((driver_standings == "\\N").sum())
# print((lap_times == "\\N").sum())
# print((pit_stops == "\\N").sum())
# print((seasons == "\\N").sum())
# print((status == "\\N").sum())


Drivers:
driverId         0
driverRef        0
number         802
code           757
forename         0
surname          0
dob              0
nationality      0
url              0
dtype: int64

Races:
raceId            0
year              0
round             0
circuitId         0
name              0
date              0
time            731
url               0
fp1_date       1035
fp1_time       1057
fp2_date       1035
fp2_time       1057
fp3_date       1053
fp3_time       1072
quali_date     1035
quali_time     1057
sprint_date    1107
sprint_time    1110
dtype: int64

Results:
resultId               0
raceId                 0
driverId               0
constructorId          0
number                 6
grid                   0
position           10916
positionText           0
positionOrder          0
points                 0
laps                   0
time               18931
milliseconds       18931
fastestLap         18478
rank               18249
fastestLapTime     18478
fastestLapSpeed

In [22]:
# Stworzenie zmiennej z punktami/rankingiem teamu przed danym wyścigiem

constructor_standings = constructor_standings.sort_values(by='raceId')
constructor_standings['points_before'] = constructor_standings.groupby('constructorId')['points'].shift(1)
constructor_standings['ranking_before'] = constructor_standings.groupby('constructorId')['position'].shift(1)
constructor_standings.dropna(inplace=True)
constructor_standings['points_before']=constructor_standings['points_before'].astype(int)
constructor_standings['ranking_before']=constructor_standings['ranking_before'].astype(int)
constructor_standings = constructor_standings[['raceId','constructorId','points_before','ranking_before']]

In [23]:
constructor_standings

,raceId,constructorId,points_before,ranking_before
3676,2,9,0,8
3684,2,23,18,1
3682,2,7,11,2
3675,2,6,0,9
3680,2,3,3,4
...,...,...,...,...
13205,1126,117,40,5
13206,1126,15,0,10
13207,1126,210,5,7
13208,1126,215,7,6


In [24]:
# Stworzenie zmiennej z punktami/rankingiem kierowcy przed danym wyścigiem

driver_standings = driver_standings.sort_values(by='raceId')
driver_standings['points_before'] = driver_standings.groupby('driverId')['points'].shift(1)
driver_standings['ranking_before'] = driver_standings.groupby('driverId')['position'].shift(1)
driver_standings.dropna(inplace=True)
driver_standings['points_before']=driver_standings['points_before'].astype(int)
driver_standings['ranking_before']=driver_standings['ranking_before'].astype(int)

In [25]:
# Stworzenie zmiennej ze średnim czasem trwania pit-stopu w sezonie dla poszczególnych teamów

pit_stops_merged = pd.merge(pit_stops, results, on = ['driverId','raceId'], how = 'left')
pit_stops_merged = pd.merge(pit_stops_merged, races, on = ['raceId'], how = 'left')
pit_stops_merged = pit_stops_merged[['raceId','constructorId','duration','year']]
pit_stops_merged['duration'] = pd.to_numeric(pit_stops_merged['duration'], errors='coerce')
pit_stops_merged['avg_season_duration'] = pit_stops_merged.groupby(['constructorId', 'year'])['duration'].transform('mean')
pit_stops_merged = pit_stops_merged[['year','constructorId','avg_season_duration']]
pit_stops_merged = pit_stops_merged.drop_duplicates()

In [26]:
pit_stops_merged.loc[pit_stops_merged['year']==2024].sort_values(by='constructorId')

,year,constructorId,avg_season_duration
10578,2024,1,22.434800
10550,2024,3,23.691211
10552,2024,6,21.888706
10554,2024,9,22.328842
10547,2024,15,26.523905
10548,2024,117,22.919609
10551,2024,131,22.246316
10546,2024,210,23.899478
10549,2024,214,24.166955
10560,2024,215,22.021375


In [27]:
# Łączenie zbiorów i usuwanie niepotrzebnych/zduplikowanych kolumn

merged_df = pd.merge(results, status, on = 'statusId', how = 'left')
merged_df = pd.merge(merged_df, drivers, on = 'driverId', how = 'left')
merged_df = pd.merge(merged_df, races, on = 'raceId', how = 'left')
merged_df = pd.merge(merged_df, constructors, on = 'constructorId', how = 'left')
merged_df = pd.merge(merged_df, driver_standings, on = ['driverId','raceId'], how = 'left')
merged_df = pd.merge(merged_df, pit_stops_merged, on = ['year','constructorId'], how = 'left')
merged_df = pd.merge(merged_df, constructor_standings, on = ['raceId','constructorId'], how = 'left')
merged_df = merged_df.drop(['driverRef','number_y','number_x','code','dob','nationality_x','url_x','round'
                            ,'time_y','url_y','constructorRef','nationality_y','url','fp1_date'
                           ,'fp1_time','fp2_date','fp2_time','fp3_date','fp3_time','quali_date'
                           ,'quali_time','sprint_date','sprint_time','position_x','fastestLap'
                           ,'fastestLapTime','rank','fastestLapSpeed','driverStandingsId'
                             ,'points_y','position_y','positionText_y','wins'], axis = 1)

In [28]:
# Uporządkowanie kolumn i zmiana nazw
order = ['resultId','raceId','name_x','year','date','driverId','forename','surname','constructorId','name_y','avg_season_duration','grid'
        ,'positionText_x','positionOrder','points_x','points_before_x','ranking_before_x','points_before_y','ranking_before_y','laps','time_x','milliseconds'
        ,'statusId','status','circuitId']
merged_df = merged_df[order]
merged_df = merged_df.rename(columns={'name_x':'race_name','date':'race_date','name_y':'constructor_name','grid':'start_position'
                                     ,'time_x':'time','rank':'fastest_lap_rank','positionOrder':'end_position','points_before_x':'driver_ranking_points_before_race'
                                     ,'ranking_before_x':'driver_ranking_before_race','positionText_x':'end_positionText','points_x':'points','avg_season_duration':'avg_season_pit_stop_duration'
                                     ,'ranking_before_y':'constructor_ranking_before_race','points_before_y':'constructor_ranking_points_before_race'})

In [29]:
(merged_df == "\\N").sum()

resultId                                      0
raceId                                        0
race_name                                     0
year                                          0
race_date                                     0
driverId                                      0
forename                                      0
surname                                       0
constructorId                                 0
constructor_name                              0
avg_season_pit_stop_duration                  0
start_position                                0
end_positionText                              0
end_position                                  0
points                                        0
driver_ranking_points_before_race             0
driver_ranking_before_race                    0
constructor_ranking_points_before_race        0
constructor_ranking_before_race               0
laps                                          0
time                                    

In [30]:
merged_df['status'].value_counts()
# Można pomyśleć czy bierzemy tylko Finished

status
Finished          7463
+1 Lap            3944
Engine            2022
+2 Laps           1600
Accident          1057
                  ... 
Launch control       1
Tyre puncture        1
+26 Laps             1
CV joint             1
Crankshaft           1
Name: count, Length: 137, dtype: int64

In [31]:
merged_df = merged_df.loc[merged_df['status']=='Finished']
merged_df = merged_df.loc[merged_df['year']>=2014]
merged_df = merged_df.loc[merged_df['start_position']!=0] #
merged_df.dropna(inplace=True)
merged_df['driver_ranking_points_before_race'] = merged_df['driver_ranking_points_before_race'].astype(int)
merged_df['driver_ranking_before_race'] = merged_df['driver_ranking_before_race'].astype(int)
merged_df['constructor_ranking_points_before_race'] = merged_df['constructor_ranking_points_before_race'].astype(int)
merged_df['constructor_ranking_before_race'] = merged_df['constructor_ranking_before_race'].astype(int)
(merged_df == "\\N").sum()

resultId                                  0
raceId                                    0
race_name                                 0
year                                      0
race_date                                 0
driverId                                  0
forename                                  0
surname                                   0
constructorId                             0
constructor_name                          0
avg_season_pit_stop_duration              0
start_position                            0
end_positionText                          0
end_position                              0
points                                    0
driver_ranking_points_before_race         0
driver_ranking_before_race                0
constructor_ranking_points_before_race    0
constructor_ranking_before_race           0
laps                                      0
time                                      0
milliseconds                              0
statusId                        

In [32]:
# for col in merged_df.columns:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2157 entries, 22127 to 26397
Data columns (total 25 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   resultId                                2157 non-null   int64  
 1   raceId                                  2157 non-null   int64  
 2   race_name                               2157 non-null   object 
 3   year                                    2157 non-null   int64  
 4   race_date                               2157 non-null   object 
 5   driverId                                2157 non-null   int64  
 6   forename                                2157 non-null   object 
 7   surname                                 2157 non-null   object 
 8   constructorId                           2157 non-null   int64  
 9   constructor_name                        2157 non-null   object 
 10  avg_season_pit_stop_duration            2157 non-null   floa

In [33]:
merged_df = merged_df.sort_values(by=['driverId', 'raceId'])

# Przesunięcie kolumny end_position, aby nie uwzględniać bieżącego wyścigu
merged_df['shifted_end_position'] = merged_df.groupby('driverId')['end_position'].shift(1)

# Obliczenie średniej pozycji końcowej z ostatnich 5 wyścigów (bez obecnego wyścigu)
merged_df['avg_last_5_end_positions'] = merged_df.groupby('driverId')['shifted_end_position'].transform(lambda x: x.rolling(5, min_periods=1).mean())

# Usunięcie kolumny pomocniczej
merged_df = merged_df.drop(columns=['shifted_end_position'])

In [37]:
merged_df.dropna(inplace=True)

In [17]:
# merged_df.isna().sum()
# (merged_df == "\\N").sum()

In [18]:
# profile = ydata_profiling.ProfileReport(merged_df, title="Profiling Report")
# profile.to_file("output_new.html")

In [38]:
merged_df

,resultId,raceId,race_name,year,race_date,driverId,forename,surname,constructorId,constructor_name,avg_season_pit_stop_duration,start_position,end_positionText,end_position,points,driver_ranking_points_before_race,driver_ranking_before_race,constructor_ranking_points_before_race,constructor_ranking_before_race,laps,time,milliseconds,statusId,status,circuitId,avg_last_5_end_positions
22171,22174,902,Bahrain Grand Prix,2014,2014-04-06,1,Lewis,Hamilton,131,Mercedes,24.296973,2,1,1,25.0,25,2,68,1,57,1:39:42.743,5982743,1,Finished,3,1.0
22193,22196,903,Chinese Grand Prix,2014,2014-04-20,1,Lewis,Hamilton,131,Mercedes,24.296973,1,1,1,25.0,50,2,111,1,54,1:33:28.338,5608338,1,Finished,17,1.0
22215,22218,904,Spanish Grand Prix,2014,2014-05-11,1,Lewis,Hamilton,131,Mercedes,24.296973,1,1,1,25.0,75,2,154,1,66,1:41:05.155,6065155,1,Finished,4,1.0
22238,22241,905,Monaco Grand Prix,2014,2014-05-25,1,Lewis,Hamilton,131,Mercedes,24.296973,2,2,2,18.0,100,1,197,1,78,+9.210,6576871,1,Finished,6,1.0
22282,22285,907,Austrian Grand Prix,2014,2014-06-22,1,Lewis,Hamilton,131,Mercedes,24.296973,9,2,2,18.0,118,2,258,1,71,+1.932,5276908,1,Finished,70,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26189,26195,1116,United States Grand Prix,2023,2023-10-22,858,Logan,Sargeant,3,Williams,25.846513,16,10,10,1.0,0,20,23,7,56,+1:27.998,5809360,1,Finished,69,14.2
26255,26261,1119,Las Vegas Grand Prix,2023,2023-11-19,858,Logan,Sargeant,3,Williams,25.846513,6,16,16,0.0,1,21,28,7,50,+50.882,5399171,1,Finished,80,13.0
26275,26281,1120,Abu Dhabi Grand Prix,2023,2023-11-26,858,Logan,Sargeant,3,Williams,25.846513,20,16,16,0.0,1,21,28,7,58,+1:27.791,5310415,1,Finished,24,14.0
26375,26381,1125,Chinese Grand Prix,2024,2024-04-21,858,Logan,Sargeant,3,Williams,23.691211,20,17,17,0.0,0,21,0,8,56,+1:35.110,6147664,1,Finished,17,13.8


In [59]:
columns_of_interest = ['avg_season_pit_stop_duration', 'start_position', 
                       'driver_ranking_before_race', 'constructor_ranking_before_race', 'end_position', 'avg_last_5_end_positions']
selected_df = merged_df[columns_of_interest]

# Konwersja wybranych kolumn na one-hot encoding
start_position_dummies = pd.get_dummies(selected_df['start_position'], prefix='start_position', dtype=int)
driver_ranking_dummies = pd.get_dummies(selected_df['driver_ranking_before_race'], prefix='driver_ranking', dtype=int)
constructor_ranking_dummies = pd.get_dummies(selected_df['constructor_ranking_before_race'], prefix='constructor_ranking', dtype=int)

# Scalanie wszystkiego w nowym DataFrame
new_df = pd.concat([selected_df['avg_season_pit_stop_duration'],
                    selected_df['avg_last_5_end_positions'],
                    start_position_dummies, 
                    driver_ranking_dummies, 
                    constructor_ranking_dummies, 
                    selected_df['end_position']], axis=1)

In [60]:
new_df['in_top_5'] = (new_df['end_position'] <= 5).astype(int)
new_df.drop('end_position',axis=1, inplace=True)

In [61]:
new_df

,avg_season_pit_stop_duration,avg_last_5_end_positions,start_position_1,start_position_2,start_position_3,start_position_4,start_position_5,start_position_6,start_position_7,start_position_8,start_position_9,start_position_10,start_position_11,start_position_12,start_position_13,start_position_14,start_position_15,start_position_16,start_position_17,start_position_18,start_position_19,start_position_20,start_position_21,start_position_22,driver_ranking_1,driver_ranking_2,driver_ranking_3,driver_ranking_4,driver_ranking_5,driver_ranking_6,driver_ranking_7,driver_ranking_8,driver_ranking_9,driver_ranking_10,driver_ranking_11,driver_ranking_12,driver_ranking_13,driver_ranking_14,driver_ranking_15,driver_ranking_16,driver_ranking_17,driver_ranking_18,driver_ranking_19,driver_ranking_20,driver_ranking_21,driver_ranking_22,driver_ranking_23,constructor_ranking_1,constructor_ranking_2,constructor_ranking_3,constructor_ranking_4,constructor_ranking_5,constructor_ranking_6,constructor_ranking_7,constructor_ranking_8,constructor_ranking_9,constructor_ranking_10,constructor_ranking_11,in_top_5
22171,24.296973,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
22193,24.296973,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
22215,24.296973,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
22238,24.296973,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
22282,24.296973,1.2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26189,25.846513,14.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
26255,25.846513,13.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
26275,25.846513,14.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
26375,23.691211,13.8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [69]:
from sklearn.model_selection import train_test_split

X = new_df.drop(columns=['in_top_5'])
y = new_df['in_top_5']

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [77]:
X

,avg_season_pit_stop_duration,avg_last_5_end_positions,start_position_1,start_position_2,start_position_3,start_position_4,start_position_5,start_position_6,start_position_7,start_position_8,start_position_9,start_position_10,start_position_11,start_position_12,start_position_13,start_position_14,start_position_15,start_position_16,start_position_17,start_position_18,start_position_19,start_position_20,start_position_21,start_position_22,driver_ranking_1,driver_ranking_2,driver_ranking_3,driver_ranking_4,driver_ranking_5,driver_ranking_6,driver_ranking_7,driver_ranking_8,driver_ranking_9,driver_ranking_10,driver_ranking_11,driver_ranking_12,driver_ranking_13,driver_ranking_14,driver_ranking_15,driver_ranking_16,driver_ranking_17,driver_ranking_18,driver_ranking_19,driver_ranking_20,driver_ranking_21,driver_ranking_22,driver_ranking_23,constructor_ranking_1,constructor_ranking_2,constructor_ranking_3,constructor_ranking_4,constructor_ranking_5,constructor_ranking_6,constructor_ranking_7,constructor_ranking_8,constructor_ranking_9,constructor_ranking_10,constructor_ranking_11
22171,24.296973,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
22193,24.296973,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
22215,24.296973,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
22238,24.296973,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
22282,24.296973,1.2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26189,25.846513,14.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
26255,25.846513,13.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
26275,25.846513,14.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
26375,23.691211,13.8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=1000)

# Trenowanie modelu
model.fit(X_train, y_train)

# Przewidywanie klas na zbiorze testowym
y_pred = model.predict(X_test)

# Obliczenie dokładności modelu
accuracy = accuracy_score(y_test, y_pred)
print(f'Dokładność modelu: {accuracy:.2f}')

Dokładność modelu: 0.86


In [72]:
avg_last_5 = merged_df.loc[(merged_df['year']==2024)&(merged_df['raceId']==1126)].sort_values(by='driver_ranking_before_race')

In [73]:
merged_df_2024 = merged_df[merged_df['year'] == 2024]

# Wyliczenie pogrupowanych danych - avg pit stop
grouped_data = merged_df_2024.groupby(['constructor_name', 'driverId', 'surname'])['avg_season_pit_stop_duration'].mean().reset_index()
grouped_data

,constructor_name,driverId,surname,avg_season_pit_stop_duration
0,Alpine F1 Team,839,Ocon,24.166955
1,Alpine F1 Team,842,Gasly,24.166955
2,Aston Martin,4,Alonso,22.919609
3,Aston Martin,840,Stroll,22.919609
4,Ferrari,832,Sainz,21.888706
5,Ferrari,844,Leclerc,21.888706
6,Haas F1 Team,807,Hülkenberg,23.899478
7,Haas F1 Team,825,Magnussen,23.899478
8,McLaren,846,Norris,22.434800
9,McLaren,857,Piastri,22.434800


In [74]:
new_df.head()

,avg_season_pit_stop_duration,avg_last_5_end_positions,start_position_1,start_position_2,start_position_3,start_position_4,start_position_5,start_position_6,start_position_7,start_position_8,start_position_9,start_position_10,start_position_11,start_position_12,start_position_13,start_position_14,start_position_15,start_position_16,start_position_17,start_position_18,start_position_19,start_position_20,start_position_21,start_position_22,driver_ranking_1,driver_ranking_2,driver_ranking_3,driver_ranking_4,driver_ranking_5,driver_ranking_6,driver_ranking_7,driver_ranking_8,driver_ranking_9,driver_ranking_10,driver_ranking_11,driver_ranking_12,driver_ranking_13,driver_ranking_14,driver_ranking_15,driver_ranking_16,driver_ranking_17,driver_ranking_18,driver_ranking_19,driver_ranking_20,driver_ranking_21,driver_ranking_22,driver_ranking_23,constructor_ranking_1,constructor_ranking_2,constructor_ranking_3,constructor_ranking_4,constructor_ranking_5,constructor_ranking_6,constructor_ranking_7,constructor_ranking_8,constructor_ranking_9,constructor_ranking_10,constructor_ranking_11,in_top_5
22171,24.296973,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
22193,24.296973,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
22215,24.296973,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
22238,24.296973,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
22282,24.296973,1.2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [75]:
avg_last_5[['surname','avg_last_5_end_positions']]

,surname,avg_last_5_end_positions
26380,Verstappen,1.0
26382,Pérez,2.8
26381,Leclerc,3.4
26383,Sainz,3.6
26379,Norris,4.8
26391,Piastri,6.4
26386,Russell,5.4
26387,Alonso,7.0
26384,Hamilton,8.6
26395,Stroll,9.2


In [79]:
# Stworzenie zbioru danych dla każdego kierowcy ze zmiennymi avg_season_pit_stop_duration, start_position, driver_ranking, constructor_ranking

drivers_data = {
    'Verstappen': [22.328842, #avg_season_pit_stop_duration
                   1.0, #avg_last_5_end_positions
                   1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, #start_position
                   1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  #driver_ranking
                   1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #constructor_ranking
    'Perez': [22.328842,
                2.8,
                0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'Leclerc': [21.888706,
                 3.4,
                 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'Norris': [22.434800,
                 4.8,
                 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    'Sainz': [21.888706,
                 3.6,
                 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'Piastri': [22.434800,
                 6.4,
                 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    'Russell': [22.246316,
                 5.4,
                 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    'Alonso': [22.919609,
                 7.0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    'Hamilton': [22.246316,
                 8.6,
                 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    'Tsunoda': [22.021375,
                 8.8,
                 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    'Stroll': [22.919609,
                 9.2,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    'Magnussen': [23.899478,
                 13.2,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 
                 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
#     'Bearman': [21.888706,
#                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
#                  0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'Hulkenberg': [23.899478,
                   11.4,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    'Ricciardo': [22.021375,
                  11.4,
                 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    'Ocon': [24.166955,
                 8.8,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 
                 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    'Albon': [23.691211,
                 11.6,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 
                 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    'Zhou': [26.523905,
                 13.8,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    'Gasly': [24.166955,
                 11.0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 
                 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    'Bottas': [26.523905,
                 12.4,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    'Sargeant': [23.691211,
                 16.0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 
                 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
    
}
drivers_df = pd.DataFrame.from_dict(drivers_data, orient='index')
drivers_df
# Obliczenie prawdopodobieństwa znalezienia się kierowcy w pierwszej 5
probabilities = model.predict_proba(drivers_df)[:, 1]

# Dodanie kolumny z nazwiskami kierowców
drivers_df['probability_in_top_5'] = probabilities
drivers_df['driver'] = drivers_df.index

sorted_drivers_df = drivers_df.sort_values(by='probability_in_top_5', ascending=False).reset_index(drop=True)

# Przypisywanie pozycji końcowej na podstawie posortowanego prawdopodobieństwa
sorted_drivers_df['predicted_end_position'] = sorted_drivers_df.index + 1
print(sorted_drivers_df[['driver', 'probability_in_top_5', 'predicted_end_position']])

        driver  probability_in_top_5  predicted_end_position
0   Verstappen              0.960279                       1
1        Sainz              0.947496                       2
2      Leclerc              0.890799                       3
3       Norris              0.808446                       4
4        Perez              0.796402                       5
5      Piastri              0.678751                       6
6      Russell              0.434844                       7
7     Hamilton              0.150952                       8
8       Alonso              0.141771                       9
9    Ricciardo              0.123327                      10
10     Tsunoda              0.116887                      11
11        Ocon              0.075513                      12
12       Gasly              0.047703                      13
13        Zhou              0.029875                      14
14      Stroll              0.020590                      15
15      Bottas          

C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
